# Introduction to deterministic and stochastic modelling of simple biological systems: part 2
## Dr Marc Sturrock

## Dual reporter system

We end by simulating the classical dual reporter study by Elowitz et al. using Biomodelling.jl.
First we call on the Biomodelling package. The biomodelling package simulates the full stochastic simulation algorithm as well as Tau-leaping approximations and also keeps track of an exponentially growing population of cells including cell division and partitioning. 

In [ ]:
using Biomodelling, Statistics, Plots
gr(legend=:bottomleft)

We will begin by simulating the low intrinsic noise case. For Poisson processes, the noise level is proportional to the reciprocal of the mean, i.e.,
$\frac{1}{mean(GFP)}$ and since $mean(GFP) = \frac{b}{d}$ for a birth death process (where $b$ is the birth rate and $d$ is the death rate), the noise level can be written as $\frac{d}{b}$. Hence if we make $b$ large and $d$ small, the noise level decreases.

In [ ]:
# define the model parameteres, reactions and initial condition
b = 1000.0
d = .1
reaction1 = (name = "birth of gfp", rate = b, reactants = [:NULL], products =[:GFP] , coeff_rea = [1] , coeff_pro = [1] )
reaction2 = (name = "gfp decay", rate = d, reactants = [:GFP], products =[:NULL], coeff_rea = [1], coeff_pro = [1])
reaction3 = (name = "birth of rfp", rate = b, reactants = [:NULL], products =[:RFP] , coeff_rea = [1] , coeff_pro = [1] )
reaction4 = (name = "rfp decay", rate = d, reactants = [:RFP], products =[:NULL], coeff_rea = [1], coeff_pro = [1])
intracellular_reactions = (reaction1, reaction2, reaction3, reaction4)
initial_conditions = [:NULL 0;:GFP 1000;:RFP 1000]
model = Biomodelling.Donne(intracellular_reactions,initial_conditions,100.0,0.1,20,0.03,0.03);

In [ ]:
#set division noise to zero
div_noise = 0.0;

In [ ]:
# simulate these intracellular reactions within an exponentially growing cell population

t_low, V_low, output_low = Biomodelling.exponential_growth(model,div_noise,ssa);

First by choosing a high birth rate and low death rate with the division noise set to zero, there is no intrinsic or extrsinsic noise, so the same gene expression and cell growth rate can be observed in every cell. We can confirm this by plotting the data for ten cells.

In [ ]:
plot(t_low,V_low[:,1:10],xlabel="time",ylabel="cell volume",label=["cell $i" for i in 1:10])

In [ ]:
plot(t_low,output_low[:,1:10,2],xlabel="t",ylabel="GFP",label=["cell $i" for i in 1:10])

In [ ]:
plot(t_low,output_low[:,1:10,3],xlabel="t",ylabel="RFP",label=["cell $i" for i in 1:10])

We can also reproduce the scatter plots shown in Elowitz et al. We will use a normalised scatter plot which we can then use to compare with different conditions.

In [ ]:
scatter(output_low[end,:,2]./maximum(output_low[end,:,2]),output_low[end,:,3]./maximum(output_low[end,:,2]),label="",xlabel="GFP",ylabel="RFP",markersize=15)

Let's compute the correlation between the GFP and RFP molecules and display it in the title of the previously made plot.

In [ ]:
title!("correlation = $(cor(output_low[end,:,2],output_low[end,:,3]))")

We next repeat this simulation with a higher level of extrinsic noise (in the form of assymetric cell division)

In [ ]:
# define the model parameteres, reactions and initial condition
b = 1000.0
d = .1
reaction1 = (name = "birth of gfp", rate = b, reactants = [:NULL], products =[:GFP] , coeff_rea = [1] , coeff_pro = [1] )
reaction2 = (name = "gfp decay", rate = d, reactants = [:GFP], products =[:NULL], coeff_rea = [1], coeff_pro = [1])
reaction3 = (name = "birth of rfp", rate = b, reactants = [:NULL], products =[:RFP] , coeff_rea = [1] , coeff_pro = [1] )
reaction4 = (name = "rfp decay", rate = d, reactants = [:RFP], products =[:NULL], coeff_rea = [1], coeff_pro = [1])
intracellular_reactions = (reaction1, reaction2, reaction3, reaction4)
initial_conditions = [:NULL 0;:GFP 1000;:RFP 1000]
model = Biomodelling.Donne(intracellular_reactions,initial_conditions,100.0,0.1,20,0.03,0.03);

In [ ]:
div_noise = 0.05;

In [ ]:
# simulate these intracellular reactions within an exponentially growing cell population

t_high, V_high, output_high = Biomodelling.exponential_growth(model,div_noise,ssa);

In [ ]:
plot(t_high,V_high[:,1:10],xlabel="time",ylabel="cell volume",label="")

We use the same initial conditions for each cell, so this is why we see identical behaviour prior to the first division event. 

In [ ]:
plot(t_high,output_high[:,1:10,2],xlabel="time",ylabel="GFP",label="")

In [ ]:
plot(t_high,output_high[:,1:10,3],xlabel="time",ylabel="RFP",label="")

In [ ]:
scatter(output_high[end,:,2]./maximum(output_high[end,:,2]),output_high[end,:,3]./maximum(output_high[end,:,3]),label="",xlabel="GFP",ylabel="RFP",markersize=15)

In [ ]:
title!("correlation = $(cor(output_high[end,:,2],output_high[end,:,3]))")

We end by inviting you to show the impact of increasing intrinsic noise on this system. Hint: How would you change the values of $b$ and $d$ to increase the level of intrinsic noise in the system? How does intrinsic noise impact the correlation between GFP and RFP?